In [ ]:
# Change directory to the project folder
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/Median-Filtering-Forensics-Based-on-Convlutional-Neural-Network')

In [ ]:
import cv2
import os
import numpy as np
import random
import shutil
import glob
from PIL import Image
from sklearn.utils import shuffle
from utils import *
import tensorflow as tf
import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model

dataset_path = './data/UCID/ucid.v2'
jpeg_dataset_path = './data/UCID/JPEG70'

In [ ]:
# Testing model on whole JPEG70 and MF5-JPEG70 data
if not os.path.exists('./data/temp'):
    # Creating directories
    os.makedirs('./data/temp/JPEG70')
    os.makedirs('./data/temp/MF5-JPEG70')

    for img in os.listdir(jpeg_dataset_path):
        ## Creating negative data
        # Reading the image
        I = cv2.imread(os.path.join(jpeg_dataset_path, img), 0)
        I_crop = center_crop(I)

        # MFR-filter layer
        I_MFR = cv2.medianBlur(I_crop, 5) - I_crop

        # Saving the image
        cv2.imwrite(f'./data/temp/JPEG70/{img}', I_MFR)

        ## Creating positive data
        img_name = img.split('.')[0] + '.tif'
        # Reading the image
        I = cv2.imread(os.path.join(dataset_path, img_name), 0)
        I_crop = center_crop(I)

        # Creating the median blurred image
        I_blurred = cv2.medianBlur(I_crop, 5)

        # MFR-filter layer
        I_MFR = cv2.medianBlur(I_blurred, 5) - I_blurred

        # Saving the image
        cv2.imwrite('./data/temp/temp.png', I_MFR)

        # Reading the image and saving it in jpeg compression file
        I = Image.open('./data/temp/temp.png')
        I.save(f"./data/temp/MF5-JPEG70/{img_name.split('.')[0] + '.jpeg'}", 'JPEG', quality=70)

    os.remove('./data/temp/temp.png')

    assert len(os.listdir('./data/temp/JPEG70')) == 1338
    assert len(os.listdir('./data/temp/MF5-JPEG70')) == 1338

In [ ]:
data_path = './data/temp'

data_batches = ImageDataGenerator(rescale=1./255).flow_from_directory(directory=data_path, target_size=(64, 64),
                        color_mode='grayscale', batch_size=256, classes=['JPEG70', 'MF5-JPEG70'], shuffle=True)

Found 2676 images belonging to 2 classes.


In [ ]:
combined_2_model = load_model('./data/saved_models/combined_2_model.h5', compile=True)

In [ ]:
score, acc = combined_2_model.evaluate(data_batches, steps=data_batches.samples//data_batches.batch_size+1, verbose=0)
print(f'Score is: {score}')
print(f'Accuracy is: {acc}')

Score is: 0.024361394345760345
Accuracy is: 0.994020938873291


In [ ]:
# Removing temp files
shutil.rmtree('./data/temp/')